# Семинар 9: обработка текстовых данных

Из [курса](https://github.com/hse-ds/iad-intro-ds/tree/master)

## Вступление
Многие практические задачи так или иначе могут вовлекать в себя работу с текстовыми данными, например:

- классификация текстов
    - анализ тональности (например, позитивный/негативный отзыв)
    - фильтрация спама
    - по теме или жанру
- машинный перевод
- распознавание и синтез речи
- извлечение информации
    - именованные сущности (например, извлечение имен, локаций, названий организаций)
    - извлечение фактов и событий
- кластеризация текстов
- оптическое распознавание символов
- проверка правописания
- вопросно-ответные системы, информационный поиск
- суммаризация текстов
- генерация текстов

В целом, алгоритм работы с текстовыми данными можно разбить на такие шаги:

- предобработка сырых данных
- токенизация (создание словаря)
- обработка словаря (удаление стоп-слов и пунктуации)
- обработка токенов (лемматизация / стемминг)
- векторизация текста (bag of words, TF-IDF, etc)

Сегодня мы познакомимся с основами работы с текстовыми данными: рассмотрим некоторые методы предобработки и простейшие алгоритмы векторизации.

### План семинара
1. Токенизация
2. Стоп-слова и пунктуация
3. Лемматизация и стемминг
4. Bag-of-words и TD-IDF
5. Решение задачи с текстовыми данными
6. Немножко про категориальные признаки

## 1. Токенизация

Токенизировать — значит, поделить текст на слова, или *токены*. Самый наивный способ токенизировать текст — разделить на слова по пробелам с помощью `split`. Но `split` упускает очень много всего, например, не отделяет пунктуацию от слов. Кроме этого есть ещё много менее тривиальных проблем, которые мы обсудим по ходу семинара, поэтому на практике всегда используют готовые токенизаторы.

In [ ]:
# !pip install nltk

In [ ]:
import warnings

import nltk
import pandas as pd
from nltk.tokenize import word_tokenize

warnings.filterwarnings("ignore")

In [ ]:
nltk.download("punkt", quiet=True)

In [ ]:
example = "Но не каждый хочет что-то исправлять:("

In [ ]:
# c помощью split()
example.split()

In [ ]:
# c помощью токенизатора
word_tokenize(example)

В nltk вообще есть довольно много токенизаторов:

In [ ]:
from nltk import tokenize

dir(tokenize)[:16]

Можно получить индексы начала и конца каждого токена:

In [ ]:
wh_tok = tokenize.WhitespaceTokenizer()
list(wh_tok.span_tokenize(example))

Некторые токенизаторы ведут себя специфично:

In [ ]:
tokenize.TreebankWordTokenizer().tokenize("don't stop me")

Для некоторых задач это может быть полезно.

А некоторые предназначены вообще не для текста на естественном языке:

In [ ]:
tokenize.SExprTokenizer().tokenize("(a (b c)) d e (f)")

Есть токенизатор, который может быть полезен для работы с твитами или сообщениями из соц. сетей. Он сохранит смайлики, хештеги и т.п.

In [ ]:
from nltk.tokenize import TweetTokenizer

tw = TweetTokenizer()
tw.tokenize(example)

## 2. Стоп-слова и пунктуация

*Стоп-слова* — это слова, которые часто встречаются практически в любом тексте и ничего интересного не говорят о конретном документе, то есть играют роль шума. Поэтому их принято убирать. По той же причине убирают и пунктуацию.

In [ ]:
nltk.download("stopwords", quiet=True)

In [ ]:
from nltk.corpus import stopwords

print(stopwords.words("russian"))

In [ ]:
print(stopwords.words("english"))

In [ ]:
from string import punctuation

punctuation

In [ ]:
noise = stopwords.words("russian") + list(punctuation)

## 3. Лемматизация и стемминг
### 3.1 Лемматизация

[**Лемматизация**](https://en.wikipedia.org/wiki/Lemmatisation) — процесс приведения слова к его нормальной форме (**лемме**):
- для существительных — именительный падеж, единственное число;
- для прилагательных — именительный падеж, единственное число, мужской род;
- для глаголов, причастий, деепричастий — глагол в инфинитиве.

Например, токены «пью», «пил», «пьет» перейдут в «пить». Почему это хорошо?
* Во-первых, мы хотим рассматривать как отдельный признак каждое *слово*, а не каждую его отдельную форму.
* Во-вторых, некоторые стоп-слова стоят только в начальной форме, и без лемматизации выкидываем мы только её.

Для русского есть два хороших лемматизатора: `mystem` и `pymorphy`.

#### [Mystem](https://tech.yandex.ru/mystem/)

Mystem — это пример популярной библиотеки для лемматизации. Как с ним работать:
* скачать mystem и запускать [из терминала с разными параметрами](https://tech.yandex.ru/mystem/doc/)
* использовать обёртку для питона [pymystem3](https://pythonhosted.org/pymystem3/pymystem3.html) (она медленнее, но удобнее в использовании)

In [ ]:
# !pip install pymystem3

In [ ]:
from pymystem3 import Mystem

mystem_analyzer = Mystem()

Мы инициализировали Mystem c дефолтными параметрами. А вообще параметры есть такие:
* mystem_bin — путь к `mystem`, если их несколько
* grammar_info — нужна ли грамматическая информация или только леммы (по умолчанию нужна)
* disambiguation — нужно ли снятие [омонимии](https://ru.wikipedia.org/wiki/%D0%9E%D0%BC%D0%BE%D0%BD%D0%B8%D0%BC%D1%8B) - дизамбигуация (по умолчанию нужна)
* entire_input — нужно ли сохранять в выводе все (пробелы, например), или можно выкинуть (по умолчанию оставляется все)

Методы Mystem принимают строку, токенизатор вшит внутри. Можно, конечно, и пословно анализировать, но тогда он не сможет учитывать контекст.

Можно просто лемматизировать текст:

In [ ]:
print(mystem_analyzer.lemmatize(example))

### [Pymorphy](http://pymorphy2.readthedocs.io/en/latest/)
Это модуль на питоне, довольно быстрый и с кучей функций.

In [ ]:
!pip install pymorphy2

In [ ]:
from pymorphy2 import MorphAnalyzer

In [ ]:
pymorphy2_analyzer = MorphAnalyzer()

pymorphy2 работает с отдельными словами. Если дать ему на вход предложение, то он его просто не лемматизирует, т.к. не понимает.

Метод MorphAnalyzer.parse() принимает слово и возвращает все возможные его разборы.

У каждого разбора есть тег. Тег — это набор граммем, характеризующих данное слово. Например, тег 'VERB,perf,intr plur,past,indc' означает, что слово — глагол (VERB) совершенного вида (perf), непереходный (intr), множественного числа (plur), прошедшего времени (past), изъявительного наклонения (indc).


In [ ]:
ana = pymorphy2_analyzer.parse("хочет")
ana

In [ ]:
ana[0].normal_form

### mystem vs. pymorphy

1) *Надеемся, что вы пользуетесь линуксом или маком* — mystem работает невероятно медленно под windows на больших текстах

2) *Снятие омонимии*. Mystem умеет снимать омонимию по контексту (хотя не всегда преуспевает), pymorphy2 берет на вход одно слово и соответственно вообще не умеет дизамбигуировать по контексту.

In [ ]:
homonym1 = "За время обучения я прослушал больше сорока курсов."
homonym2 = "Сорока своровала блестящее украшение со стола."

# корректно определил части речи
# NUM — числительное
# S — существительное
print(mystem_analyzer.analyze(homonym1)[-5])
print(mystem_analyzer.analyze(homonym2)[0])

### 3.2 Стемминг

В отличие от лемматизации, при применении стемминга у всех слов отбрасываются аффиксы (окончания и суффиксы), что необязательно приводит слова к формам, существующим в рассматриваемом языке. [**Snowball**](http://snowball.tartarus.org/) — фрэймворк для написания алгоритмов стемминга. Алгоритмы стемминга отличаются для разных языков и используют знания о конкретном языке: списки окончаний для разных частей речи, разных склонений и т.д.

In [ ]:
from nltk.stem.snowball import SnowballStemmer

In [ ]:
tokenized_example = word_tokenize(example)

In [ ]:
stemmer = SnowballStemmer("russian")
stemmed_example = [stemmer.stem(w) for w in tokenized_example]
print(" ".join(stemmed_example))

Для английского получится что-то такое.

In [ ]:
text = 'In my younger and more vulnerable years my father gave me some advice that I\'ve been turning over in my mind ever since.\n"Whenever you feel like criticizing any one," he told me, "just remember that all the people in this world haven\'t had the advantages that you\'ve had."'
print(text)
text_tokenized = [w for w in word_tokenize(text) if w.isalpha()]
print("==========")
print(text_tokenized)

In [ ]:
stemmer = SnowballStemmer("english")
text_stemmed = [stemmer.stem(w) for w in text_tokenized]
print(" ".join(text_stemmed))

## 4. Bag-of-words и TF-IDF

Но как же все-таки работать с текстами, используя стандартные методы машинного обучения? Ведь нам нужны объекты выборки, которые описываются числами, а не словами. Иначе говоря, нам нужно *векторизовать* текстовые данные.

### 4.1 Bag-of-words

Пусть у нас имеется коллекция текстов $D = \{d_i\}_{i=1}^l$ и словарь всех слов, встречающихся в выборке $V = \{v_j\}_{j=1}^d.$ В этом случае некоторый текст $d_i$ описывается вектором $(x_{ij})_{j=1}^d,$ где
$$x_{ij} = \sum_{v \in d_i} [v = v_j].$$

Таким образом, текст $d_i$ описывается вектором количества вхождений каждого слова из словаря в данный текст.

In [ ]:
texts = [
    "I like my cat.",
    "My cat is the most perfect cat.",
    "is this cat or is this bread?",
]

In [ ]:
texts_tokenized = [
    " ".join([w for w in word_tokenize(t) if w.isalpha()]) for t in texts
]
texts_tokenized

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

cnt_vec = CountVectorizer()
X = cnt_vec.fit_transform(texts_tokenized)

In [ ]:
cnt_vec.vocabulary_.keys()

In [ ]:
X

In [ ]:
X.toarray()

In [ ]:
pd.DataFrame(X.toarray(), columns=cnt_vec.get_feature_names_out())

### 4.2 TF-IDF

Заметим, что если слово часто встречается в одном тексте, но почти не встречается в других, то оно получает для данного текста большой вес, ровно так же, как и слова, которые часто встречаются в каждом тексте. Для того чтобы разделять эти такие слова, можно использовать статистическую меру TF-IDF, характеризующую важность слова для конкретного текста. Для каждого слова из текста $d$ рассчитаем относительную частоту встречаемости в нем (Term Frequency):
$$
\text{TF}(t, d) = \frac{C(t | d)}{\sum\limits_{k \in d}C(k | d)},
$$
где $C(t | d)$ - число вхождений слова $t$ в текст $d$.

Также для каждого слова из текста $d$ рассчитаем обратную частоту встречаемости в корпусе текстов $D$ (Inverse Document Frequency):
$$
\text{IDF}(t, D) = \log\left(\frac{|D|}{|\{d_i \in D \mid t \in d_i\}|}\right)
$$
Логарифмирование здесь проводится с целью уменьшить масштаб весов, ибо зачастую в корпусах присутствует очень много текстов.

В итоге каждому слову $t$ из текста $d$ теперь можно присвоить вес
$$
\text{TF-IDF}(t, d, D) = \text{TF}(t, d) \times \text{IDF}(t, D)
$$
Интерпретировать формулу выше несложно: действительно, чем чаще данное слово встречается в данном тексте и чем реже в остальных, тем важнее оно для этого текста.

Отметим, что в качестве TF и IDF можно использовать другие [определения](https://en.wikipedia.org/wiki/Tf%E2%80%93idf#Definition).

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vec = TfidfVectorizer()
X = tfidf_vec.fit_transform(texts_tokenized)

In [ ]:
tfidf_vec.vocabulary_.keys()

In [ ]:
X

In [ ]:
X.toarray()

In [ ]:
pd.DataFrame(X.toarray(), columns=cnt_vec.get_feature_names_out())

**Вопросик:** что изменилось по сравнению с использованием метода `CountVectorizer`?

## 5. Решение задачи с текстовыми данными

Будем решать задачу классификации твитов по тональности. Возьмём датасет из твитов, в котором про каждый твит известно, как он эмоционально окрашен: положительно или отрицательно. Задача: предсказывать эмоциональную окраску. Классификацию по тональности используют, например, в рекомендательных системах, чтобы понять, понравилось ли людям кафе, кино, etc.

Скачиваем куски датасета ([источник](http://study.mokoron.com/)): [положительные](https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv?dl=0), [отрицательные](https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv).

In [ ]:
# если у вас линукс / мак / collab или ещё какая-то среда, в которой работает wget, можно так:
!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MaxAbsScaler

In [ ]:
# считываем данные и заполняем общий датасет
positive = pd.read_csv("positive.csv", sep=";", usecols=[3], names=["text"])
positive["label"] = "positive"
negative = pd.read_csv("negative.csv", sep=";", usecols=[3], names=["text"])
negative["label"] = "negative"
df = pd.concat([positive, negative])

In [ ]:
df.tail()

In [ ]:
df.shape

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df.text, df.label, random_state=13)

#### n-граммы

n-граммы — это последовательности n токенов из исходного текста. В простейшем случае это могут быть последовательности из букв или последовательности из слов. Давайте посмотрим подробнее на примере.

In [ ]:
from nltk import ngrams

In [ ]:
sent = "Если б мне платили каждый раз".split()
list(ngrams(sent, 1))  # униграммы

In [ ]:
list(ngrams(sent, 2))  # биграммы

In [ ]:
list(ngrams(sent, 3))  # триграммы

In [ ]:
list(ngrams(sent, 5))  # ... пентаграммы?

В качестве альтернативы можно пользоваться `CountVectorizer`, который работает так:
* строит для каждого документа (каждой пришедшей ему строки) вектор размерности количества токенов в нашем словаре
* заполняет каждый i-тый элемент количеством вхождений токена в данный документ

Параметр `ngram_range` отвечает за то, какие n-граммы мы используем в качестве фичей:
- `ngram_range=(1, 1)` — униграммы
- `ngram_range=(3, 3)` — триграммы
- `ngram_range=(1, 3)` — униграммы, биграммы и триграммы

### 5.1 Обучение моделей

Давайте обучим наш первый бейзлайн — логрег на униграммах!

In [ ]:
vec = CountVectorizer(ngram_range=(1, 1))
bow = vec.fit_transform(x_train)  # bow — bag of words (мешок слов)
bow_test = vec.transform(x_test)

scaler = MaxAbsScaler()
bow = scaler.fit_transform(bow)
bow_test = scaler.transform(bow_test)

clf = LogisticRegression(max_iter=200, random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(bow_test)
print(classification_report(y_test, pred))

Попробуем сделать то же самое для триграмм.

In [ ]:
vec = CountVectorizer(ngram_range=(3, 3))
bow = vec.fit_transform(x_train)
bow_test = vec.transform(x_test)

scaler = MaxAbsScaler()
bow = scaler.fit_transform(bow)
bow_test = scaler.transform(bow_test)

clf = LogisticRegression(max_iter=200, random_state=42)
clf.fit(bow, y_train)
pred_thrgramm = clf.predict(bow_test)
print(classification_report(y_test, pred_thrgramm))

В нашем случае стало хуже :)

А теперь повторим процедуру для TF-IDF.

In [ ]:
vec = TfidfVectorizer(ngram_range=(1, 1))
vec_train = vec.fit_transform(x_train)
vec_test = vec.transform(x_test)

scaler = MaxAbsScaler()
vec_train = scaler.fit_transform(vec_train)
vec_test = scaler.transform(vec_test)

clf = LogisticRegression(max_iter=300, random_state=42)
clf.fit(vec_train, y_train)
pred_tfidf = clf.predict(vec_test)
print(classification_report(y_test, pred_tfidf))

### 5.2 О важности эксплоративного анализа

Но иногда пунктуация бывает и не шумом. Главное — отталкиваться от задачи. Что будет если вообще не убирать пунктуацию?

In [ ]:
vec = CountVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize)
bow = vec.fit_transform(x_train)
bow_test = vec.transform(x_test)

scaler = MaxAbsScaler()
bow = scaler.fit_transform(bow)
bow_test = scaler.transform(bow_test)

clf = LogisticRegression(max_iter=200, random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(bow_test)
print(classification_report(y_test, pred))

Стоило оставить пунктуацию, и внезапно все метрики устремились к 1. Как это получилось? Среди неё были очень значимые токены (как вы думаете, какие?). Найдём признак с самыми большим коэффициентом:

In [ ]:
vec.get_feature_names_out()[np.argmax(clf.coef_)]

Посмотрим, как один из супер-значительных токенов справится с классификацией безо всякого машинного обучения:

In [ ]:
cool_token = vec.get_feature_names_out()[np.argmax(clf.coef_)]
pred = ["positive" if cool_token in tweet else "negative" for tweet in x_test]
print(classification_report(y_test, pred))

In [ ]:
cool_token = vec.get_feature_names_out()[np.argmax(clf.coef_)]
tweets_with_cool_token = [tweet for tweet in x_train if cool_token in tweet]
np.random.seed(42)
for tweet in np.random.choice(tweets_with_cool_token, size=10, replace=False):
    print(tweet)

### 5.3 Символьные n-граммы

Теперь в качестве признаков используем, например, униграммы символов:

In [ ]:
vec = CountVectorizer(ngram_range=(1, 1), analyzer="char")
bow = vec.fit_transform(x_train)
bow_test = vec.transform(x_test)

scaler = MaxAbsScaler()
bow = scaler.fit_transform(bow)
bow_test = scaler.transform(bow_test)

clf = LogisticRegression(max_iter=200, random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(bow_test)
print(classification_report(y_test, pred))

В общем-то, теперь уже понятно, почему на этих данных здесь 1. Так или иначе, на символах классифицировать тоже можно: для некоторых задач (например, для определения языка) признаки-символьные n-граммы могут внести серьезный вклад в качество модели. Ещё одна замечательная особенность признаков-символов: токенизация и лемматизация не нужна, можно использовать такой подход для языков, у которых нет готовых анализаторов.

## 6. Немножко про категориальные признаки

Существует три основных способа обработки категориальных значений:
- Label encoding
- One-hot-кодирование
- Счётчики (CTR, mean-target кодирование) — каждый категориальный признак заменяется на среднее значение целевой переменной по всем объектам, имеющим одинаковое значение в этом признаке

Сравним два последних метода наборе данных Титаник (предсказание выживших пассажиров).

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
data = pd.read_csv(
    "https://raw.githubusercontent.com/iad34/seminars/master/materials/data_sem1.csv",
    sep=";",
)

data["Age"] = data["Age"].fillna(data.groupby("Pclass")["Age"].transform("median"))
data.drop("Cabin", axis=1, inplace=True)
data.drop("Name", axis=1, inplace=True)
data.drop("Ticket", axis=1, inplace=True)
data.dropna(inplace=True)

### 6.1 OHE

In [ ]:
data_ohe = pd.get_dummies(data, drop_first=True)

y = data_ohe["Survived"]
data_ohe = data_ohe.drop(["Survived"], axis=1)
data_ohe.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    data_ohe, y, test_size=0.2, random_state=42, shuffle=True
)

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = LogisticRegression(max_iter=300)
model.fit(X_train, y_train)
prediction = model.predict(X_test)
print(f"ROC AUC: {roc_auc_score(prediction, y_test):.2f}")

### 6.2 Mean target encoding

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    data, y, test_size=0.2, random_state=42, shuffle=True
)

In [ ]:
mean_target = X_train.groupby("Sex")["Survived"].mean()
X_train.loc[:, "Sex"] = X_train["Sex"].replace(mean_target)
X_test.loc[:, "Sex"] = X_test["Sex"].replace(mean_target)

mean_target_e = X_train.groupby("Embarked")["Survived"].mean()
X_train.loc[:, "Embarked"] = X_train["Embarked"].replace(mean_target_e)
X_test.loc[:, "Embarked"] = X_test["Embarked"].replace(mean_target_e)

X_train.drop(["Survived"], axis=1, inplace=True)
X_test.drop(["Survived"], axis=1, inplace=True)

In [ ]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = LogisticRegression(max_iter=300)
model.fit(X_train, y_train)
prediction = model.predict(X_test)
print(f"ROC AUC: {roc_auc_score(prediction, y_test):.2f}")

Кодирование признаков с помощью счетчиков приведённом выше может приводить к переобучению. Почему?

Чтобы бороться с этим, можно экспериментировать с разными модификациями:

1. Вычислять значение счётчика по всем объектам расположенным выше в датасете (например, если у нас выборка отсортирована по времени)
2. Вычислять по фолдам, то есть делить выборку на некоторое количество частей и подсчитывать значение признаков по всем фолдам кроме текущего (как делается в кросс-валидации)
3. Добавлять шум в посчитанные признаки